# QHO Calculator 

This python programme is designed to compute the expectation values of the quantum harmonic oscillator states. This deals with the ladder operator, bra-ket approach. We note the position, momentum operators $ \hat{x}, \hat{y} $ in terms of ladder operators 

$$ \hat{x} = \sqrt{\frac{\hbar}{2m\omega}} (\hat{a_+} + \hat{a_-}) $$
$$ $$
$$ \hat{p} = i\sqrt{\frac{\hbar m\omega}{2}} (\hat{a_+} - \hat{a_-}) $$

We also note the important results of kets, $ |n> $

$$ \hat{a_+}|n> = \sqrt{n+1}|n+1> $$
$$ \hat{a_-}|n> = \sqrt{n}|n-1> $$

In [1]:
import numpy as np 

In [2]:
#coefficients corresponding ket states 
c = np.array([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)])
n = np.array([0,1,2])

In [3]:
def a_plus(coefficients, ket_states):
    
    """This function in takes the original |Ψ> in terms of its superposition of |n> states and
       outputs the new coefficients, ket states of a_+|Ψ>
       Inputs: Coefficients (coefficients in the superposition), corresponding ket states |n>
       Outputs: Coefficients, kets states of a_+|Ψ> (2 arrays)"""
    
    #initialize coefficients, ket states output for a+ acting on |Ψ>
    c_a_plus = np.array([])
    n_a_plus = np.array([])
    
    #loop to calculate coefficients, ket states for a_+|Ψ>
    for i in ket_states:
    
        k_n_raised = np.sqrt(i+1)
        c_n_raised = coefficients[np.where(ket_states==i)[0][0]]*k_n_raised
        n_raised = i+1
    
        c_a_plus = np.append(c_a_plus, c_n_raised)
        n_a_plus = np.append(n_a_plus, n_raised)
    
    return c_a_plus, n_a_plus

def a_minus(coefficients, ket_states):
    
    """This function in takes the original |Ψ> in terms of its superposition of |n> states and
       outputs the new coefficients, ket states of a_-|Ψ>
       Inputs: Coefficients (coefficients in the superposition), corresponding ket states |n>
       Outputs: Coefficients, kets states of a_-|Ψ> (2 arrays)"""
    
    #initialize coefficients, ket states output for a- acting on |Ψ>
    c_a_minus = np.array([])
    n_a_minus = np.array([])
    
    #loop to calculate coefficients, ket states for a_-|Ψ>
    for i in ket_states: 
        
        # i > 0 since a_-|0> = 0 (we also avoid sqrt(i) when i is negative)
        if (i > 0) == True:
            k_n_lowered = np.sqrt(i)
            c_n_lowered = coefficients[np.where(ket_states==i)[0][0]]*k_n_lowered
            n_lowered = i-1
    
            c_a_minus = np.append(c_a_minus, c_n_lowered)
            n_a_minus = np.append(n_a_minus, n_lowered)
        
    return c_a_minus, n_a_minus

In [4]:
def position_operator(coefficients, ket_states):
    
    """This function calculuates x̂|Ψ>, outputs the resulting coefficients and ket states.
       We do this by computing (a_+ + a_-)|Ψ> and then multiplying the factor of 1/√2. Note that this omits 
       the factor of √(ℏ/mω)
       
       Inputs: Coefficients (coefficients in the superposition), corresponding ket states |n>
       Outputs: Coefficients, ket states of x̂|Ψ> (up to a factor of √(ℏ/mω)) """
    
    #initializes arrays for coefficients, ket states of (a_+ + a_-)|Ψ>
    c_a_plus_a = np.array([])
    n_a_plus_a = np.array([])
    
    #two loops to combine a_+|Ψ>, a_-|Ψ> to compute (a_+ + a_-)|Ψ>
    for i in a_minus(coefficients, ket_states)[1]:
        
        #computes common kets of a_+|Ψ>, a_-|Ψ>, adds them together
        if (i in a_plus(coefficients, ket_states)[1]) == True:
            
            #get coefficients corresponding to given state 
            c_plus = a_plus(coefficients, ket_states)[0][np.where(a_plus(coefficients, ket_states)[1]==i)[0][0]]
            c_minus = a_minus(coefficients, ket_states)[0][np.where(a_minus(coefficients, ket_states)[1]==i)[0][0]]
            
            #adds together coefficients of a_+, a_-
            c_a_plus_a = np.append(c_a_plus_a, c_plus+c_minus)
            n_a_plus_a = np.append(n_a_plus_a, i)
        
        #appends ket states unique to a_-|Ψ>, which do not combine with a_+|Ψ>
        else:
            
            c_minus = a_minus(coefficients, ket_states)[0][np.where(a_minus(coefficients, ket_states)[1]==i)[0][0]]
            c_a_plus_a = np.append(c_a_plus_a, c_minus)
            n_a_plus_a = np.append(n_a_plus_a, i)  

    for i in a_plus(coefficients, ket_states)[1]:
        
        #appends ket states unique to a_+|Ψ>, which do not combine with a_-|Ψ>
        if (i in a_minus(coefficients, ket_states)[1]) == False:
            
            c_plus = a_plus(coefficients, ket_states)[0][np.where(a_plus(coefficients, ket_states)[1]==i)[0][0]]
            c_a_plus_a = np.append(c_a_plus_a, c_plus)
            n_a_plus_a = np.append(n_a_plus_a, i)
    
    #multiply by 1/√2 to get position ket coefficients (up to factor √(ℏ/mω))
    return c_a_plus_a / np.sqrt(2), n_a_plus_a

def momentum_operator(coefficients, ket_states):
    
    """This function calculuates p̂|Ψ>, outputs the resulting coefficients and ket states.
       We do this by computing (a_+ - a_-)|Ψ> and then multiplying the factor of 1/√2. Note that this omits 
       the factor of √(ℏ/mω). 
       
       Inputs: Coefficients (coefficients in the superposition), corresponding ket states |n>
       Outputs: Coefficients, ket states of p̂|Ψ> (up to a factor of √(ℏmω)) """
    
    c_a_minus_a = np.array([])
    n_a_minus_a = np.array([])
    
    #two loops to combine a_+|Ψ>, a_-|Ψ> to compute (a_+ - a_-)|Ψ>
    for i in a_minus(coefficients, ket_states)[1]:
        
        #computes common kets of a_+|Ψ>, a_-|Ψ>, subtracts one from the other
        if (i in a_plus(coefficients, ket_states)[1]) == True:
            
            c_plus = a_plus(coefficients, ket_states)[0][np.where(a_plus(coefficients, ket_states)[1]==i)[0][0]]
            c_minus = a_minus(coefficients, ket_states)[0][np.where(a_minus(coefficients, ket_states)[1]==i)[0][0]]
            #subtracts coefficients of a_-|Ψ> from a_+|Ψ>
            c_a_minus_a = np.append(c_a_minus_a, c_plus-c_minus)
            n_a_minus_a = np.append(n_a_minus_a, i)
            
        else:
            #appends ket states unique to a_-|Ψ>, which do not combine with a_+|Ψ>
            c_minus = a_minus(coefficients, ket_states)[0][np.where(a_minus(coefficients, ket_states)[1]==i)[0][0]]
            
            #coefficients given factor of -1  (- a_-|Ψ>)
            c_a_minus_a = np.append(c_a_minus_a, -1*c_minus)
            n_a_minus_a = np.append(n_a_minus_a, i)  

    for i in a_plus(coefficients, ket_states)[1]:
        
        #appends ket states unique to a_+|Ψ>, which do not combine with a_-|Ψ>
        if (i in a_minus(coefficients, ket_states)[1]) == False:
            
            c_plus = a_plus(coefficients, ket_states)[0][np.where(a_plus(coefficients, ket_states)[1]==i)[0][0]]
            c_a_minus_a = np.append(c_a_minus_a, c_plus)
            n_a_minus_a = np.append(n_a_minus_a, i)
    
    #multiply by 1/√2 to get momentum ket coefficients (up to factor √(ℏ/mω))       
    return 1j*c_a_minus_a / np.sqrt(2), n_a_minus_a


In [5]:
def position_squared_operator(coefficients, ket_states):
    
    """This function calculuates x̂^2|Ψ>, outputs the resulting coefficients and ket states.
       We do this by computing (a_+ + a_-)^2|Ψ>. Note that this omits the factor of (ℏ/mω) """
    
    #apply position operator twice
    x_psi = position_operator(coefficients, ket_states)
    x_squared_psi = position_operator(x_psi[0], x_psi[1])
    
    return x_squared_psi[0], x_squared_psi[1]

def momentum_squared_operator(coefficients, ket_states):
    
    """This function calculuates x̂^2|Ψ>, outputs the resulting coefficients and ket states.
       We do this by computing (a_+ + a_-)^2|Ψ>. Note that this omits the factor of (ℏmω)"""
    
    #apply momentum operator twice 
    p_psi = momentum_operator(coefficients, ket_states)
    p_squared_psi = momentum_operator(p_psi[0], p_psi[1])
    
    return p_squared_psi[0], p_squared_psi[1]

In [6]:
def expectationvaluecalculator(coefficients, ket_states, operator):
    
    """This function calculates the expectation value of a given ket state |Ψ>, <Ψ|Q|Ψ> for 
       given operator Q """
    
    operator_output = operator(coefficients, ket_states)
    operator_coefficients = operator_output[0]
    operator_ket_states = operator_output[1]
    
    inner_product_coefficients = np.array([])
    for i in ket_states:
        
        if (i in operator_ket_states) == True:
            c_ket = coefficients[np.where(ket_states==i)[0][0]]
            c_operator = operator_coefficients[np.where(operator_ket_states==i)[0][0]]
            inner_product_coefficients = np.append(inner_product_coefficients, np.conjugate(c_ket)*c_operator)
    
    expectation_value = np.sum(inner_product_coefficients)
    
    return expectation_value
            

# Use 

To use the calculator, one must add the required states and the corresponding coefficients attached to the states. This assumes the coefficients are such that the total wavefunction is normalized. 

eg. for 

$$ | \psi \rangle = \frac{1}{\sqrt{3}} | 0 \rangle + \frac{1}{\sqrt{3}} | 1 \rangle + \frac{1}{\sqrt{3}} | 3 \rangle  $$

One would have to put the array of coefficients as 

$$ c = [ \frac{1}{\sqrt{3}}, \frac{1}{\sqrt{3}}, \frac{1}{\sqrt{3}} ] $$

and the array of states as 

$$ n = [0,1,3] $$

In [7]:
#use required wavefunction coefficients, ket states here
c = 1/np.sqrt(6) * np.array([1,2,1])
n = np.array([0,1,2])

#calculate momentum, magnitude momentum squared expectation values
p_avg = expectationvaluecalculator(c,n,momentum_operator)
p_squared_avg = expectationvaluecalculator(c,n,momentum_squared_operator)

#print expectation values 
print(f'<p> = {p_avg} √(ℏmω) ')
print(f'<p^2> = {p_squared_avg} ℏmω')

<p> = 0j √(ℏmω) 
<p^2> = (1.2642977396044843+0j) ℏmω


Using the position and momentum operator functions, one can also construct more operator functions such as the hamiltonian and obtain the expectation value. 